In [2]:
import pandas as pd
import duckdb
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print(" Librerías importadas correctamente")
print(f" Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

 Librerías importadas correctamente
 Fecha de ejecución: 2025-10-27 13:53:08


In [8]:
print("Cargando dataset Online Retail II...")

# Cargar el archivo CSV (CORRECCIÓN)
df = pd.read_csv('../data/online_retail.csv')

print("Dataset cargado exitosamente")
print(f"Dimensiones del dataset: {df.shape[0]:,} filas x {df.shape[1]} columnas")

# Mostrar información de columnas
print("\n COLUMNAS DEL DATASET:")
print(df.columns.tolist())

Cargando dataset Online Retail II...
Dataset cargado exitosamente
Dimensiones del dataset: 1,067,371 filas x 8 columnas

 COLUMNAS DEL DATASET:
['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country']


In [10]:
print("EXPLORACIÓN INICIAL DEL DATASET")
print("=" * 50)

# Información general
print("1. INFORMACIÓN GENERAL:")
print(df.info())

print("\n2. PRIMERAS FILAS:")
display(df.head(10))

print("\n3. ESTADÍSTICAS DESCRIPTIVAS:")
display(df.describe())

EXPLORACIÓN INICIAL DEL DATASET
1. INFORMACIÓN GENERAL:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB
None

2. PRIMERAS FILAS:


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085.0,United Kingdom
6,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,13085.0,United Kingdom
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,13085.0,United Kingdom
9,489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01 07:46:00,3.75,13085.0,United Kingdom



3. ESTADÍSTICAS DESCRIPTIVAS:


,Quantity,Price,Customer ID
count,1.067371e+06,1.067371e+06,824364.000000
mean,9.938898e+00,4.649388e+00,15324.638504
std,1.727058e+02,1.235531e+02,1697.464450
min,-8.099500e+04,-5.359436e+04,12346.000000
25%,1.000000e+00,1.250000e+00,13975.000000
50%,3.000000e+00,2.100000e+00,15255.000000
75%,1.000000e+01,4.150000e+00,16797.000000
max,8.099500e+04,3.897000e+04,18287.000000


In [9]:
print("VERIFICANDO ESTRUCTURA DEL CSV")
print("=" * 40)

# Verificar si hay múltiples años en el dataset
print("Valores únicos en el dataset:")
for col in df.columns:
    if df[col].dtype == 'object':
        unique_vals = df[col].nunique()
        print(f"  - {col}: {unique_vals} valores únicos")
        
# Verificar si necesitamos filtrar por año
if 'Year' in df.columns:
    print(f"\nAños en el dataset: {df['Year'].unique()}")

VERIFICANDO ESTRUCTURA DEL CSV
Valores únicos en el dataset:
  - Invoice: 53628 valores únicos
  - StockCode: 5305 valores únicos
  - Description: 5698 valores únicos
  - InvoiceDate: 47635 valores únicos
  - Country: 43 valores únicos


In [11]:
print("ANÁLISIS DE VALORES NULOS")
print("=" * 40)

null_analysis = pd.DataFrame({
    'Columna': df.columns,
    'Valores_Nulos': df.isnull().sum(),
    'Porcentaje_Nulos': (df.isnull().sum() / len(df) * 100).round(2)
})

display(null_analysis)

print(f"\n Hallazgo clave: {null_analysis[null_analysis['Valores_Nulos'] > 0]['Columna'].values} tienen valores nulos")

ANÁLISIS DE VALORES NULOS


,Columna,Valores_Nulos,Porcentaje_Nulos
Invoice,Invoice,0,0.00
StockCode,StockCode,0,0.00
Description,Description,4382,0.41
Quantity,Quantity,0,0.00
InvoiceDate,InvoiceDate,0,0.00
Price,Price,0,0.00
Customer ID,Customer ID,243007,22.77
Country,Country,0,0.00



 Hallazgo clave: ['Description' 'Customer ID'] tienen valores nulos


In [12]:
print("INICIANDO LIMPIEZA DE DATOS")
print("=" * 40)

# Guardar estado original
filas_originales = len(df)
print(f"Filas originales: {filas_originales:,}")

# 1. Eliminar filas donde CustomerID es nulo
df_clean = df[df['Customer ID'].notna()].copy()
print(f"Eliminados {filas_originales - len(df_clean):,} filas con CustomerID nulo")

# 2. Eliminar cantidades negativas (devoluciones)
filas_antes = len(df_clean)
df_clean = df_clean[df_clean['Quantity'] > 0]
print(f"Eliminadas {filas_antes - len(df_clean):,} filas con Quantity ≤ 0 (devoluciones)")

# 3. Eliminar precios cero o negativos
filas_antes = len(df_clean)
df_clean = df_clean[df_clean['Price'] > 0]
print(f"Eliminadas {filas_antes - len(df_clean):,} filas con Price ≤ 0")

# 4. Convertir Customer ID a entero
df_clean['Customer ID'] = df_clean['Customer ID'].astype(int)

# 5. Crear columna de ingreso total
df_clean['Total_Revenue'] = df_clean['Quantity'] * df_clean['Price']

print(f"\n RESUMEN LIMPIEZA:")
print(f"   - Filas originales: {filas_originales:,}")
print(f"   - Filas después de limpieza: {len(df_clean):,}")
print(f"   - Filas eliminadas: {filas_originales - len(df_clean):,}")
print(f"   - Porcentaje conservado: {(len(df_clean)/filas_originales*100):.1f}%")

INICIANDO LIMPIEZA DE DATOS
Filas originales: 1,067,371
Eliminados 243,007 filas con CustomerID nulo
Eliminadas 18,744 filas con Quantity ≤ 0 (devoluciones)
Eliminadas 71 filas con Price ≤ 0

 RESUMEN LIMPIEZA:
   - Filas originales: 1,067,371
   - Filas después de limpieza: 805,549
   - Filas eliminadas: 261,822
   - Porcentaje conservado: 75.5%


In [17]:
print("CONFIGURANDO DUCKDB PARA ANÁLISIS SQL")
print("=" * 50)

# Crear conexión DuckDB
con = duckdb.connect()

# Registrar DataFrame limpio como tabla SQL
con.register('ventas', df_clean)

print("Conexión DuckDB establecida")
print("Tabla 'ventas' registrada para consultas SQL")

# Probar con consulta básica
result = con.execute("""
    SELECT 
        COUNT(*) as total_filas,
        COUNT(DISTINCT "Customer ID") as clientes_unicos,
        MIN(InvoiceDate) as fecha_inicio,
        MAX(InvoiceDate) as fecha_fin,
        SUM(Total_Revenue) as ingreso_total
    FROM ventas
""").fetchdf()

display(result)

CONFIGURANDO DUCKDB PARA ANÁLISIS SQL
Conexión DuckDB establecida
Tabla 'ventas' registrada para consultas SQL


,total_filas,clientes_unicos,fecha_inicio,fecha_fin,ingreso_total
0,805549,5878,2009-12-01 07:45:00,2011-12-09 12:50:00,1.774343e+07


Análisis Exploratorio SQL

In [20]:
print("ANÁLISIS EXPLORATORIO CON SQL")
print("=" * 45)

#  Top 10 clientes por gasto
print("TOP 10 CLIENTES POR GASTO:")
top_clientes = con.execute("""
    SELECT 
        "Customer ID" as cliente_id,
        COUNT(*) as total_transacciones,
        SUM(Total_Revenue) as gasto_total,
        AVG(Total_Revenue) as ticket_promedio
    FROM ventas
    GROUP BY "Customer ID"
    ORDER BY gasto_total DESC
    LIMIT 10
""").fetchdf()
display(top_clientes)

ANÁLISIS EXPLORATORIO CON SQL
TOP 10 CLIENTES POR GASTO:


,cliente_id,total_transacciones,gasto_total,ticket_promedio
0,18102,1058,608821.65,575.445794
1,14646,3849,528602.52,137.335027
2,14156,4048,313946.37,77.555921
3,14911,11245,295972.63,26.320376
4,17450,424,246973.09,582.483703
5,13694,1525,196482.81,128.841187
6,17511,1911,175603.55,91.890921
7,16446,3,168472.50,56157.500000
8,16684,718,147142.77,204.934220
9,12415,926,144458.37,156.002559


## Análisis de Recencia, Frecuencia y Valor Monetario (RFM), centrado en el valor monetario
**total_transacciones: Frecuencia (F)**


**gasto_total: Valor Monetario (M)**



**ticket_promedio: Gasto promedio por transacción**

In [24]:
# Evolución temporal de ventas
print("EVOLUCIÓN TEMPORAL DE VENTAS:")
ventas_mensuales = con.execute("""
    SELECT 
        DATE_TRUNC('month', CAST(InvoiceDate AS DATE)) as mes,  
        COUNT(*) as transacciones,
        SUM(Total_Revenue) as ingreso_mensual,
        COUNT(DISTINCT "Customer ID") as clientes_activos
    FROM ventas
    GROUP BY mes  
    ORDER BY mes
""").fetchdf()
display(ventas_mensuales)

EVOLUCIÓN TEMPORAL DE VENTAS:


,mes,transacciones,ingreso_mensual,clientes_activos
0,2009-12-01,30754,686654.160,955
1,2010-01-01,21775,557319.062,720
2,2010-02-01,23367,506371.066,772
3,2010-03-01,32301,699608.991,1057
4,2010-04-01,27235,594609.192,942
5,2010-05-01,28644,599985.790,966
6,2010-06-01,31188,639066.580,1041
7,2010-07-01,27032,591636.740,928
8,2010-08-01,26392,604242.650,911
9,2010-09-01,34599,831615.001,1145


##  Análisis de la Evolución Temporal 
**Objetivo:** Medir el rendimiento del negocio mes a mes.

 **`ingreso_mensual`**: Total de ingresos por mes (Métrica principal de rendimiento).


**`transacciones`**: Conteo de pedidos (Mide el volumen de actividad).


**`clientes_activos`**: Conteo de clientes únicos por mes (Mide la actividad de la base de clientes).

In [25]:
#  Análisis por país
print("ANÁLISIS POR PAÍS:")
ventas_pais = con.execute("""
    SELECT 
        Country as pais,
        COUNT(*) as transacciones,
        SUM(Total_Revenue) as ingreso_total,
        COUNT(DISTINCT "Customer ID") as clientes_unicos
    FROM ventas
    GROUP BY Country
    ORDER BY ingreso_total DESC
    LIMIT 15
""").fetchdf()
display(ventas_pais)

ANÁLISIS POR PAÍS:


,pais,transacciones,ingreso_total,clientes_unicos
0,United Kingdom,725250,1.472315e+07,5350
1,EIRE,15743,6.216311e+05,5
2,Netherlands,5088,5.542323e+05,22
3,Germany,16694,4.312625e+05,107
4,France,13812,3.552575e+05,95
5,Australia,1812,1.699681e+05,15
6,Spain,3719,1.091785e+05,41
7,Switzerland,3011,1.003653e+05,22
8,Sweden,1319,9.154972e+04,19
9,Denmark,798,6.986219e+04,12


## Análisis Geográfico: Rendimiento por País 
**Objetivo:** Identificar los 15 países que más contribuyen al ingreso total del negocio.

**`pais`**: El país de origen de las transacciones (Agrupación).


**`ingreso_total`**: Suma total de los ingresos generados por cada país (Clave de ordenamiento).


**`clientes_unicos`**: Número de clientes distintos en cada país (Mide la base de clientes).

In [26]:
#Verificación Final de Estructura de Datos
print(" VERIFICACIÓN FINAL DE ESTRUCTURA DE DATOS")
print("=" * 50)

# Mostrar información detallada de columnas
print("📋 COLUMNAS DISPONIBLES EN EL DATASET LIMPIO:")
for i, col in enumerate(df_clean.columns, 1):
    dtype = df_clean[col].dtype
    null_count = df_clean[col].isnull().sum()
    unique_count = df_clean[col].nunique()
    print(f"{i:2d}. {col:20} | {str(dtype):10} | Nulos: {null_count:4} | Únicos: {unique_count:6}")

# Identificar columnas clave automáticamente
print("\n IDENTIFICACIÓN AUTOMÁTICA DE COLUMNAS CLAVE:")
columnas_clave = {}
for col in df_clean.columns:
    col_lower = col.lower()
    if 'customer' in col_lower or 'id' in col_lower:
        columnas_clave['customer_id'] = col
    elif 'invoice' in col_lower and 'date' in col_lower:
        columnas_clave['invoice_date'] = col
    elif 'quantity' in col_lower:
        columnas_clave['quantity'] = col
    elif 'price' in col_lower:
        columnas_clave['price'] = col
    elif 'invoice' in col_lower:
        columnas_clave['invoice_id'] = col

print("Columnas clave identificadas:")
for key, value in columnas_clave.items():
    print(f"  - {key}: '{value}'")

 VERIFICACIÓN FINAL DE ESTRUCTURA DE DATOS
📋 COLUMNAS DISPONIBLES EN EL DATASET LIMPIO:
 1. Invoice              | object     | Nulos:    0 | Únicos:  36969
 2. StockCode            | object     | Nulos:    0 | Únicos:   4631
 3. Description          | object     | Nulos:    0 | Únicos:   5283
 4. Quantity             | int64      | Nulos:    0 | Únicos:    437
 5. InvoiceDate          | object     | Nulos:    0 | Únicos:  34585
 6. Price                | float64    | Nulos:    0 | Únicos:    665
 7. Customer ID          | int64      | Nulos:    0 | Únicos:   5878
 8. Country              | object     | Nulos:    0 | Únicos:     41
 9. Total_Revenue        | float64    | Nulos:    0 | Únicos:   3919

 IDENTIFICACIÓN AUTOMÁTICA DE COLUMNAS CLAVE:
Columnas clave identificadas:
  - invoice_id: 'Invoice'
  - quantity: 'Quantity'
  - invoice_date: 'InvoiceDate'
  - price: 'Price'
  - customer_id: 'Customer ID'


In [29]:
df_clean.to_csv('../data/online_retail_clean.csv', index=False)
print("Datos limpios guardados para reutilizar")

Datos limpios guardados para reutilizar
